## Load packages and data

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('nassauCountyUpdate.csv', low_memory=False)
school = pd.read_csv('schoolDistrictDistribution.csv', header=None)
print('The number of rows: '+str(len(data)))

The number of rows: 30845


## Update the school district numbers

In [4]:
data['school_dis'] = school
data = data.drop('School District', axis=1)

## Combine the columns with similar information

In [5]:
data['fireplace'] = data['Fireplace'].combine_first(data['Fireplace Number'])

In [6]:
data['garage'] = data['Garage'].combine_first(data['Garage Description'])

In [7]:
data['stories'] = data['Stories'].combine_first(data['Levels or Stories'])

In [8]:
data['heat'] = data['Heating'].combine_first(data['Heat']).combine_first(data['Heating Features'])

In [9]:
data['cooling'] = data['Cooling'].combine_first(data['Cooling Features'])

In [10]:
data['prop_type'] = data['Property type'].combine_first(data['Property Subtype'])

In [11]:
data['rooms'] = data['Rooms'].combine_first(data['bedrooms'])

In [12]:
data['lot_area'] = data['lot_area'].combine_first(data['lot_sizes']).combine_first(data['Lot Size Square Feet'])

In [13]:
data = data.drop(['price history', 'newpPiceHistory', #inaccurate info
                  'Fireplace', 'Fireplace Number',
                  'Garage', 'Garage Description',
                  'Stories', 'Levels or Stories',
                  'Heating', 'Heat', 'Heating Features',
                  'Cooling', 'Cooling Features',
                  'Property type', 'Property Subtype',
                  'Rooms', 'bedrooms',
                  'lot_sizes', 'lot_sizes', 'Lot Size Square Feet'], axis=1)


## Drop columns with # na values >5000

In [14]:
df = data.dropna(thresh=5000, axis=1)

df.columns

Index(['property_id', 'borough', 'latitude', 'longitude', 'house_number',
       'street', 'zipcode', 'city', 'living_area', 'lot_area', 'document_id',
       'document_amount', 'doc_date', 'key', 'Date updated', 'House size',
       'Style', 'Units', 'year built', 'tax_2016', 'tax_2015', 'school_dis',
       'fireplace', 'garage', 'stories', 'heat', 'cooling', 'prop_type',
       'rooms'],
      dtype='object')

## Drop duplicated rows

In [15]:
df.drop_duplicates(inplace=True)
len(df)

C:\Users\lllin\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


30758

## Drop irrelevant columns

In [16]:
df = df.drop(['property_id', 'document_id', 'borough', 'house_number',
       'street', 'key', 'Units', 'Date updated'], axis =1)
df.columns

Index(['latitude', 'longitude', 'zipcode', 'city', 'living_area', 'lot_area',
       'document_amount', 'doc_date', 'House size', 'Style', 'year built',
       'tax_2016', 'tax_2015', 'school_dis', 'fireplace', 'garage', 'stories',
       'heat', 'cooling', 'prop_type', 'rooms'],
      dtype='object')

## Transform document date type

In [17]:
df['doc_date'].apply(type).value_counts()

<class 'str'>    30758
Name: doc_date, dtype: int64

In [18]:
df['doc_date'] = pd.to_datetime(df['doc_date'])

In [19]:
df['doc_date'].apply(type).value_counts()

<class 'pandas._libs.tslibs.timestamps.Timestamp'>    30758
Name: doc_date, dtype: int64

## Missing values

In [20]:
df.isnull().sum().sort_values(ascending=False)

fireplace          18614
garage              5619
Style               1306
stories             1276
year built          1223
House size           964
cooling              947
prop_type            945
living_area          497
tax_2015             216
tax_2016             190
heat                 151
lot_area              28
longitude              0
zipcode                0
city                   0
rooms                  0
document_amount        0
doc_date               0
school_dis             0
latitude               0
dtype: int64

### combine living area & house size

In [21]:
# https://pbpython.com/currency-cleanup.html

df['House size'].apply(type).value_counts()

<class 'str'>      29794
<class 'float'>      964
Name: House size, dtype: int64

In [22]:
df['living_area'].apply(type).value_counts()

<class 'float'>    30758
Name: living_area, dtype: int64

In [23]:
df['House size'] = pd.to_numeric(df['House size'].apply(lambda x: x.replace(' sq ft', '').replace('sq ft', '').replace(',', '') 
                                          if isinstance(x, str) else x))

In [24]:
df['House size'].apply(type).value_counts()

<class 'float'>    30758
Name: House size, dtype: int64

In [25]:
df['House size'].isnull().sum()

1265

In [26]:
df.loc[df['House size'].isnull(), ['living_area','House size']]

,living_area,House size
0,1025.0,NaN
2,3050.0,NaN
3,1265.0,NaN
4,1464.0,NaN
5,1907.0,NaN
...,...,...
30580,1019.0,NaN
30658,1592.0,NaN
30671,1993.0,NaN
30710,1436.0,NaN


In [27]:
df['House size'].fillna(df['living_area'], inplace = True)

In [28]:
df['House size'].isnull().sum()

142

In [29]:
df = df.drop('living_area', axis =1)

### Amenities

#### cooling

In [30]:
df['cooling'].apply(type).value_counts()

<class 'str'>      29811
<class 'float'>      947
Name: cooling, dtype: int64

In [31]:
df['cooling'] = df['cooling'].apply(lambda x: 'Unknown' if type(x) == float else x)

In [32]:
df['cooling'] = df['cooling'].apply(lambda x: 0 if x == 'Unknown' else 1)

In [33]:
# 1 - has cooling; 0 - unknown
df['cooling'].value_counts()

1    29468
0     1290
Name: cooling, dtype: int64

#### fireplace

In [34]:
df['fireplace'].apply(type).value_counts()

<class 'float'>    18982
<class 'str'>      11776
Name: fireplace, dtype: int64

In [35]:
df['fireplace'] = df['fireplace'].apply(lambda x: 1 if isinstance(x, str) else x)

In [36]:
df['fireplace'] = df['fireplace'].apply(lambda x: 1 if x > 0 else 0)

In [37]:
# 1 - has fireplac; 0 - unknown
df['fireplace'].value_counts()

0    18614
1    12144
Name: fireplace, dtype: int64

#### heating

In [38]:
df['heat'].value_counts()

Central       29378
Gas             539
Unknown         348
Oil             279
Not Inc          36
Forced Air       19
Elec              3
None              2
Inc               2
Radiator          1
Name: heat, dtype: int64

In [39]:
df['heat'].apply(type).value_counts()

<class 'str'>      30607
<class 'float'>      151
Name: heat, dtype: int64

In [40]:
df['heat'] = df['heat'].apply(lambda x: 0 if isinstance(x, float) else x)

In [41]:
df['heat'] = df['heat'].apply(lambda x: 0 if x == 'Unknown' else x)

In [42]:
df['heat'] = df['heat'].apply(lambda x: -1 if x == 'Not Inc' or x == 'None' else (x if x == 0 else 1))

In [43]:
# 1 - has heat; 0 - unknown; -1 - not have heat
df['heat'].value_counts()

 1    30221
 0      499
-1       38
Name: heat, dtype: int64

#### garage

In [44]:
df['garage'].value_counts()

Garage             20284
Basement Garage     4269
Attached             331
Detached             215
Driveway               7
N                      4
Con                    4
Street                 3
Non                    3
Yes, Built-in          3
A                      2
Ind                    2
Private                1
2                      1
Gar, Drwy              1
Yes                    1
Y                      1
Private St             1
Bsm                    1
2 Car Grg!             1
Carport                1
At                     1
Tdm                    1
Pvt                    1
Name: garage, dtype: int64

In [45]:
df['garage'].apply(type).value_counts()

<class 'str'>      25139
<class 'float'>     5619
Name: garage, dtype: int64

In [46]:
df['garage'] = df['garage'].apply(lambda x: 0 if isinstance(x, float) else x)

In [47]:
df['garage'] = df['garage'].apply(lambda x: -1 if x == 'N' or x == 'Non' else (x if x == 0 else 1))

## 1 - has; 0 - unknown; -1 - not have

### Stories

In [48]:
df['stories'].value_counts()

2.0    15117
1.0    14336
3.0       28
4.0        1
Name: stories, dtype: int64

In [49]:
df['stories'].apply(type).value_counts()

<class 'float'>    30758
Name: stories, dtype: int64

In [50]:
df['stories'] = df['stories'].apply(lambda x: 0 if pd.isna(x) else x)
# 0 - unknown; else - # of stories

###  Style

In [51]:
df['Style'].value_counts()

Cape Cod           8800
Colonial           5868
Ranch              5229
Old Style          3641
Split Level        3577
Bungalow           1279
Other               318
A-Frame             225
Triplex             198
Contemporary        151
Tudor                71
Townhouse            59
Mansion              14
Victorian            10
Condo/Apartment       8
Land                  4
Name: Style, dtype: int64

In [52]:
df['Style'].apply(type).value_counts()

<class 'str'>      29452
<class 'float'>     1306
Name: Style, dtype: int64

In [53]:
df['Style'] = df['Style'].apply(lambda x: 'Unknown' if isinstance(x, float) else x)

### Year built

In [54]:
df['year built'].apply(type).value_counts()

<class 'float'>    30758
Name: year built, dtype: int64

In [55]:
df['year built'].value_counts()

1951.0    1869
1950.0    1691
1948.0    1661
1954.0    1653
1953.0    1218
          ... 
1860.0       1
1849.0       1
1842.0       1
1873.0       1
1800.0       1
Name: year built, Length: 158, dtype: int64

In [56]:
df['year built'] = df['year built'].apply(lambda x: 0 if pd.isna(x) else x)
# 0 - unknown; else - year built

### Property type

In [57]:
df['prop_type'].apply(type).value_counts()

<class 'str'>      29813
<class 'float'>      945
Name: prop_type, dtype: int64

In [58]:
df['prop_type'].value_counts()

Single family    27327
Multi family      2008
Land               443
Condo               20
condo               11
townhouse            2
co-op                1
Residential          1
Name: prop_type, dtype: int64

In [59]:
df['prop_type'] = df['prop_type'].apply(lambda x: 'Unknown' if isinstance(x, float) else x)

In [60]:
df['prop_type'] = df['prop_type'].apply(lambda x: 'Multi family' if x == 'Residential' else x)

In [61]:
df['prop_type'] = df['prop_type'].apply(lambda x: 'Condo' if x == 'condo' else x)

In [62]:
df.isnull().sum().sort_values(ascending=False)

tax_2015           216
tax_2016           190
House size         142
lot_area            28
rooms                0
Style                0
longitude            0
zipcode              0
city                 0
document_amount      0
doc_date             0
year built           0
prop_type            0
school_dis           0
fireplace            0
garage               0
stories              0
heat                 0
cooling              0
latitude             0
dtype: int64

## Write cleaned data to csv

In [63]:
df.to_csv('house_price_data_clean.csv', index=False)